# KNN_SYVM_프로모션_효율예측분석

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

In [150]:
mem = pd.read_csv("./data/member.csv")
tran = pd.read_csv("./data/transaction.csv")

* 쇼핑몰 데이터, 프로모션 쿠폰을 발행하고 사용 여부 데이터 수집
* mem: 고객 id, 최근 방문일, 사는 지역, 추천여부, 주요접속채널, 쿠폰 사용 여부(target)
* tran : 고객id, 구매수량, 총 구매 금액
* 전통적 마케팅 분석방법 RFM 기법을 활용해 고객 데이터에서 파생변수 생성 후 분석
* R: Recency: 현재일 - 최근 구매일
* F: Frequency : 구매빈도
* M: Monetary: 구매 금액
* 종속변수: conversion => 고객이 프로모션에 반응 했는가? 1=yes, 0=no

In [151]:
mem.head(2)

,id,recency,zip_code,is_referral,channel,conversion
0,906145,10,Surburban,0,Phone,0
1,184478,6,Rural,1,Web,0


In [152]:
tran.head(20)

,id,num_item,total_amount
0,906145,5,34000
1,906145,1,27000
2,906145,4,33000
3,184478,4,29000
4,394235,4,33000
5,394235,4,8000
6,130152,2,11000
7,130152,1,22000
8,130152,1,23000
9,130152,3,27000


In [153]:
mem.isna().sum()

id             0
recency        0
zip_code       0
is_referral    0
channel        0
conversion     0
dtype: int64

In [154]:
tran.isna().sum()

id              0
num_item        0
total_amount    0
dtype: int64

In [155]:
mem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           64000 non-null  int64 
 1   recency      64000 non-null  int64 
 2   zip_code     64000 non-null  object
 3   is_referral  64000 non-null  int64 
 4   channel      64000 non-null  object
 5   conversion   64000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 2.9+ MB


In [156]:
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196836 entries, 0 to 196835
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   id            196836 non-null  int64
 1   num_item      196836 non-null  int64
 2   total_amount  196836 non-null  int64
dtypes: int64(3)
memory usage: 4.5 MB


In [10]:
tran.loc[tran['id'] == 100001]

,id,num_item,total_amount
7456,100001,3,24000
7457,100001,4,28000


In [11]:
tran.value_counts()

id      num_item  total_amount
388503  1         16000           3
362633  2         25000           3
837062  1         29000           3
138844  4         17000           3
776672  1         23000           3
                                 ..
550832  3         35000           1
100113  5         26000           1
550832  5         27000           1
999881  4         24000           1
550835  1         28000           1
Name: count, Length: 194954, dtype: int64

In [60]:
total = tran.groupby(['id'])['total_amount'].agg(['sum','count']).reset_index()
total

,id,sum,count
0,100001,52000,2
1,100008,26000,1
2,100032,62000,3
3,100036,129000,5
4,100070,85000,4
...,...,...,...
63995,999932,32000,1
63996,999981,91000,4
63997,999990,84000,3
63998,999995,27000,1


In [13]:
total.columns = ['id', 'amt_sum', 'amt_count']

In [14]:
mem = pd.merge(mem, total, how='left', on= 'id')

In [26]:
mem['total_amount'] = 

In [15]:
mem

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count
0,906145,10,Surburban,0,Phone,0,94000,3
1,184478,6,Rural,1,Web,0,29000,1
2,394235,7,Surburban,1,Web,0,41000,2
3,130152,9,Rural,1,Web,0,83000,4
4,940352,2,Urban,0,Web,0,31000,1
...,...,...,...,...,...,...,...,...
63995,838295,10,Urban,0,Web,0,104000,4
63996,547316,5,Urban,1,Phone,0,89000,5
63997,131575,6,Urban,1,Phone,0,61000,2
63998,603659,1,Surburban,1,Multichannel,0,108000,5


In [72]:
mem.loc[mem['id'] == 100001]

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count
2896,100001,6,Urban,1,Phone,0,52000,2


In [24]:
mem_info = mem.groupby('conversion')['amt_sum'].agg(['mean', 'std']).mean()
mem_info

mean    73207.461958
std     32221.855832
dtype: float64

In [29]:
mem[(mem['amt_sum_zscore'] > 3) | (mem['amt_sum_zscore'] < -3)]

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count,amt_sum_zscore
14256,961395,11,Surburban,0,Phone,1,184000,5,3.438428
18389,560785,1,Surburban,1,Phone,1,171000,5,3.034975
19953,688614,1,Surburban,0,Web,1,171000,5,3.034975
29247,179940,9,Urban,1,Web,1,171000,5,3.034975
58903,861347,5,Rural,0,Phone,1,183000,5,3.407393


여기서 부터 선생님 풀이. 
z-score는 안그래도 성능이 안 좋은데 더 좋진 않았다.

In [157]:
mean_item_amount = tran.groupby('id')[['num_item', 'total_amount']].mean()
mean_item_amount.columns = [ 'mean_num_item', 'mean_total_amount']
mean_item_amount

,mean_num_item,mean_total_amount
id,,
100001,3.500000,26000.000000
100008,5.000000,26000.000000
100032,2.666667,20666.666667
100036,3.000000,25800.000000
100070,3.250000,21250.000000
...,...,...
999932,5.000000,32000.000000
999981,2.000000,22750.000000
999990,3.000000,28000.000000


In [158]:
sum_item_amount = tran.groupby('id')[['num_item', 'total_amount']].sum()
sum_item_amount.columns = ['sum_num_item', 'sum_total_amount']
sum_item_amount

,sum_num_item,sum_total_amount
id,,
100001,7,52000
100008,5,26000
100032,8,62000
100036,15,129000
100070,13,85000
...,...,...
999932,5,32000
999981,8,91000
999990,9,84000


In [159]:
freq =  tran.groupby('id')[['id']].count()
freq

,id
id,
100001,2
100008,1
100032,3
100036,5
100070,4
...,...
999932,1
999981,4
999990,3


In [160]:
mean_item_amount = mean_item_amount.join(freq)
mean_item_amount

,mean_num_item,mean_total_amount,id
id,,,
100001,3.500000,26000.000000,2
100008,5.000000,26000.000000,1
100032,2.666667,20666.666667,3
100036,3.000000,25800.000000,5
100070,3.250000,21250.000000,4
...,...,...,...
999932,5.000000,32000.000000,1
999981,2.000000,22750.000000,4
999990,3.000000,28000.000000,3


In [161]:
mean_item_amount = mean_item_amount.rename(columns={'id':'freq'})
mean_item_amount

,mean_num_item,mean_total_amount,freq
id,,,
100001,3.500000,26000.000000,2
100008,5.000000,26000.000000,1
100032,2.666667,20666.666667,3
100036,3.000000,25800.000000,5
100070,3.250000,21250.000000,4
...,...,...,...
999932,5.000000,32000.000000,1
999981,2.000000,22750.000000,4
999990,3.000000,28000.000000,3


In [162]:
tran

,id,num_item,total_amount
0,906145,5,34000
1,906145,1,27000
2,906145,4,33000
3,184478,4,29000
4,394235,4,33000
...,...,...,...
196831,536246,5,24000
196832,927617,5,26000
196833,927617,3,22000
196834,927617,3,18000


In [163]:
mean_item_amount = mean_item_amount.join(sum_item_amount)
mean_item_amount = mean_item_amount.reset_index()
mean_item_amount

,id,mean_num_item,mean_total_amount,freq,sum_num_item,sum_total_amount
0,100001,3.500000,26000.000000,2,7,52000
1,100008,5.000000,26000.000000,1,5,26000
2,100032,2.666667,20666.666667,3,8,62000
3,100036,3.000000,25800.000000,5,15,129000
4,100070,3.250000,21250.000000,4,13,85000
...,...,...,...,...,...,...
63995,999932,5.000000,32000.000000,1,5,32000
63996,999981,2.000000,22750.000000,4,8,91000
63997,999990,3.000000,28000.000000,3,9,84000
63998,999995,2.000000,27000.000000,1,2,27000


In [166]:
data =  pd.merge(mem, mean_item_amount, how='left', on='id')

----------------------
데이터 트리밍 끝


In [26]:
mem['amt_sum_zscore'] = (mem['amt_sum'] - mem_info['mean'])/mem_info['std']

In [30]:
mem_info2 = mem.groupby('conversion')['amt_count'].agg(['mean', 'std']).mean()
mem_info2

mean    3.245059
std     1.266130
dtype: float64

In [31]:
mem['amt_count_zscore'] = (mem['amt_count'] - mem_info2['mean'])/mem_info2['std']

In [32]:
mem[(mem['amt_count_zscore'] > 3) | (mem['amt_count_zscore'] < -3)]

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count,amt_sum_zscore,amt_count_zscore


In [33]:
mem

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count,amt_sum_zscore,amt_count_zscore
0,906145,10,Surburban,0,Phone,0,94000,3,0.645293,-0.193550
1,184478,6,Rural,1,Web,0,29000,1,-1.371971,-1.773166
2,394235,7,Surburban,1,Web,0,41000,2,-0.999553,-0.983358
3,130152,9,Rural,1,Web,0,83000,4,0.303910,0.596259
4,940352,2,Urban,0,Web,0,31000,1,-1.309902,-1.773166
...,...,...,...,...,...,...,...,...,...,...
63995,838295,10,Urban,0,Web,0,104000,4,0.955641,0.596259
63996,547316,5,Urban,1,Phone,0,89000,5,0.490119,1.386067
63997,131575,6,Urban,1,Phone,0,61000,2,-0.378857,-0.983358
63998,603659,1,Surburban,1,Multichannel,0,108000,5,1.079781,1.386067


In [39]:
Frequency = tran.groupby(['id'])['num_item'].count().reset_index()

In [41]:
mem = pd.merge(mem, Frequency, how='left', on= 'id')

In [42]:
mem

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count,amt_sum_zscore,amt_count_zscore,num_item
0,906145,10,Surburban,0,Phone,0,94000,3,0.645293,-0.193550,3
1,184478,6,Rural,1,Web,0,29000,1,-1.371971,-1.773166,1
2,394235,7,Surburban,1,Web,0,41000,2,-0.999553,-0.983358,2
3,130152,9,Rural,1,Web,0,83000,4,0.303910,0.596259,4
4,940352,2,Urban,0,Web,0,31000,1,-1.309902,-1.773166,1
...,...,...,...,...,...,...,...,...,...,...,...
63995,838295,10,Urban,0,Web,0,104000,4,0.955641,0.596259,4
63996,547316,5,Urban,1,Phone,0,89000,5,0.490119,1.386067,5
63997,131575,6,Urban,1,Phone,0,61000,2,-0.378857,-0.983358,2
63998,603659,1,Surburban,1,Multichannel,0,108000,5,1.079781,1.386067,5


In [43]:
obj_cols = mem[['zip_code','channel']]
num_cols = mem.select_dtypes(exclude=['object', 'category'])
obj_cols

,zip_code,channel
0,Surburban,Phone
1,Rural,Web
2,Surburban,Web
3,Rural,Web
4,Urban,Web
...,...,...
63995,Urban,Web
63996,Urban,Phone
63997,Urban,Phone
63998,Surburban,Multichannel


In [44]:
num_cols.columns

Index(['id', 'recency', 'is_referral', 'conversion', 'amt_sum', 'amt_count',
       'amt_sum_zscore', 'amt_count_zscore', 'num_item'],
      dtype='object')

In [45]:
mem.columns

Index(['id', 'recency', 'zip_code', 'is_referral', 'channel', 'conversion',
       'amt_sum', 'amt_count', 'amt_sum_zscore', 'amt_count_zscore',
       'num_item'],
      dtype='object')

In [46]:
for i in obj_cols:
    print('='*30,i,'='*30)
    print(mem[i].nunique(), mem[i].unique())
    print()
    print(mem[i].value_counts())
    print()
    print(mem.groupby(i)['conversion'].mean().sort_values(ascending=False))

============================== zip_code ==============================
3 ['Surburban' 'Rural' 'Urban']

zip_code
Surburban    28776
Urban        25661
Rural         9563
Name: count, dtype: int64

zip_code
Rural        0.188121
Surburban    0.139943
Urban        0.139044
Name: conversion, dtype: float64
============================== channel ==============================
3 ['Phone' 'Web' 'Multichannel']

channel
Web             28217
Phone           28021
Multichannel     7762
Name: count, dtype: int64

channel
Multichannel    0.171734
Web             0.159407
Phone           0.127155
Name: conversion, dtype: float64


In [47]:
for i in num_cols:
    print('='*30,i,'='*30)
    print(mem[i].describe())
    print()
    print(mem[i].value_counts())
    print()
    print(mem.groupby(i)['conversion'].mean().sort_values(ascending=False))

============================== id ==============================
count     64000.000000
mean     550694.137797
std      259105.689773
min      100001.000000
25%      326772.000000
50%      551300.000000
75%      774914.500000
max      999997.000000
Name: id, dtype: float64

id
197686    1
331595    1
448925    1
260497    1
561234    1
         ..
940352    1
130152    1
394235    1
184478    1
906145    1
Name: count, Length: 64000, dtype: int64

id
539439    1.0
941253    1.0
539390    1.0
539371    1.0
539340    1.0
         ... 
551242    0.0
999990    0.0
551265    0.0
999997    0.0
551294    0.0
Name: conversion, Length: 64000, dtype: float64
============================== recency ==============================
count    64000.000000
mean         5.763734
std          3.507592
min          1.000000
25%          2.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: recency, dtype: float64

recency
1     8952
10    7565
2     7537
9     6441
3     5904
4  

In [75]:
from ydata_profiling import ProfileReport

/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [76]:
profile = ProfileReport(mem, title="mem_EDA")

In [77]:
profile.to_file("./mem_EDA.html")

Summarize dataset:   0%| | 0/13 [00:00<?, ?it/s, Describe variable: is_referral]
Summarize dataset:  38%|▍| 5/13 [00:00<00:01,  7.42it/s, Describe variable: amt_
Export report to file: 100%|█████████████████████| 1/1 [00:00<00:00, 283.38it/s]


In [93]:
mem

,id,recency,zip_code,is_referral,channel,conversion,amt_sum,amt_count
0,906145,10,Surburban,0,Phone,0,94000,3
1,184478,6,Rural,1,Web,0,29000,1
2,394235,7,Surburban,1,Web,0,41000,2
3,130152,9,Rural,1,Web,0,83000,4
4,940352,2,Urban,0,Web,0,31000,1
...,...,...,...,...,...,...,...,...
63995,838295,10,Urban,0,Web,0,104000,4
63996,547316,5,Urban,1,Phone,0,89000,5
63997,131575,6,Urban,1,Phone,0,61000,2
63998,603659,1,Surburban,1,Multichannel,0,108000,5


In [48]:
mem.columns

Index(['id', 'recency', 'zip_code', 'is_referral', 'channel', 'conversion',
       'amt_sum', 'amt_count', 'amt_sum_zscore', 'amt_count_zscore',
       'num_item'],
      dtype='object')

In [49]:
mem = pd.get_dummies(mem, drop_first= True)

In [78]:
X = mem.drop(['conversion','id'], axis=1)
y= mem['conversion']

In [51]:
X

,id,recency,is_referral,amt_sum,amt_count,amt_sum_zscore,amt_count_zscore,num_item,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
0,906145,10,0,94000,3,0.645293,-0.193550,3,True,False,True,False
1,184478,6,1,29000,1,-1.371971,-1.773166,1,False,False,False,True
2,394235,7,1,41000,2,-0.999553,-0.983358,2,True,False,False,True
3,130152,9,1,83000,4,0.303910,0.596259,4,False,False,False,True
4,940352,2,0,31000,1,-1.309902,-1.773166,1,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
63995,838295,10,0,104000,4,0.955641,0.596259,4,False,True,False,True
63996,547316,5,1,89000,5,0.490119,1.386067,5,False,True,True,False
63997,131575,6,1,61000,2,-0.378857,-0.983358,2,False,True,True,False
63998,603659,1,1,108000,5,1.079781,1.386067,5,True,False,False,False


In [53]:
y.value_counts()

conversion
0    54606
1     9394
Name: count, dtype: int64

In [54]:
9394/(9394+54606)*100

14.678125

In [167]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score,classification_report

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4 , stratify=y , random_state=10)

In [57]:
dtc = DecisionTreeClassifier( class_weight='balanced', random_state=10 )
dtc.fit(X_train, y_train)
pred = dtc.predict(X_test)
print("=" * 30, "valid_result", "=" * 30)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

============================== valid_result ==============================
0.7755859375
              precision    recall  f1-score   support

           0       0.87      0.87      0.87     21842
           1       0.22      0.21      0.22      3758

    accuracy                           0.78     25600
   macro avg       0.54      0.54      0.54     25600
weighted avg       0.77      0.78      0.77     25600



In [58]:
lxgb = LGBMClassifier( class_weight='balanced',n_jobs=10, random_state=10 )
lxgb.fit(X_train, y_train)
pred = lxgb.predict(X_test)
print("=" * 30, "valid_result", "=" * 30)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

[LightGBM] [Info] Number of positive: 5636, number of negative: 32764
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 614
[LightGBM] [Info] Number of data points in the train set: 38400, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
============================== valid_result ==============================
0.6187109375
              precision    recall  f1-score   support

           0       0.92      0.61      0.73     21842
           1       0.23      0.69      0.35      3758

    accuracy                           0.62     25600
   macro avg       0.58      0.65      0.54     25600
weighted avg       0.82      0.62      0.67     25600



In [59]:
xgb = XGBClassifier( class_weight='balanced',n_jobs=10, random_state=10 )
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print("=" * 30, "valid_result", "=" * 30)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

============================== valid_result ==============================
0.851796875
              precision    recall  f1-score   support

           0       0.86      0.99      0.92     21842
           1       0.46      0.05      0.09      3758

    accuracy                           0.85     25600
   macro avg       0.66      0.52      0.51     25600
weighted avg       0.80      0.85      0.80     25600



/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:18:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "class_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
rfc = RandomForestClassifier( class_weight='balanced',n_jobs=10, random_state=42 )
rfc.fit(X_train, y_train)
pred = rfc.predict(X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

-----------
원핫 인코딩

In [170]:
data = pd.get_dummies(data, drop_first= True)

홀드아웃 단계

In [171]:
X = mem.drop(['conversion','id'], axis=1)
y= mem['conversion']

In [172]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4 , stratify=y , random_state=10)

# KNN, SVN

# 거리기반 알고리즘 사용시에는 독립변수들간의 단위를 꼭 맞춰주어야합니다.
# 스케일링
* Min-Max Scaler : 모든 숫자를 0-1사이의 숫자로 변환 - 데이터 분포의 모양을 그대로 유지
* Standard Scaler : 평균을 0, 표준편차를 1로 하는 정규분포 형태로 변환 -데이터의 분포 모양이 정규분포 모양으로 바꿈
* RobustScaler :사분위수를 이용하여 데이터를 스케일링 - 데이터에 이상값이 있을 때 사용, 이상값의 영향을 최소화

# 머신러닝 모델별 스케일러
* KNN(최근접이웃): MinMaxScaler, 이상치가 있는 경우 RobustScaler
* SVM(서포트 백터 머신) : StandardScalar , 이상치가 있는 경우 RobustScaler
* Losistc Regression : StandardScalar , 이상치가 있는 경우 RobustScaler
* Linear / Ridge / Lasso: StandardScalar , 이상치가 있는 경우 RobustScaler
* KMeans / DBSCAN: MinMax or StandardScalar , 이상치가 있는 경우 RobustScaler
* DecisionTree, RandomForest, XGBoost : 스케일링 불필요, 이상치가 있는 경우에도 안해도 됨.
* Naive Bayes : 스케일링 불필요, 이상치가 있는 경우에도 안해도 됨.
* 인공신경망: MinMax or StandardScalar , 이상치가 있는 경우 RobustScaler

# 스케일링 시점: train/test 나눈 후 실습

카테고리 변수를 on-hot encoding

In [174]:
data = pd.get_dummies(data, drop_first= True)

홀드아웃

In [175]:
X = data.drop(['conversion','id'], axis=1)
y= data['conversion']

In [176]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.4 , stratify=y , random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid,y_valid, test_size=0.5 , stratify=y_valid , random_state=42)

In [177]:
X

,recency,is_referral,mean_num_item,mean_total_amount,freq,sum_num_item,sum_total_amount,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
0,10,0,3.333333,31333.333333,3,10,94000,True,False,True,False
1,6,1,4.000000,29000.000000,1,4,29000,False,False,False,True
2,7,1,4.000000,20500.000000,2,8,41000,True,False,False,True
3,9,1,1.750000,20750.000000,4,7,83000,False,False,False,True
4,2,0,3.000000,31000.000000,1,3,31000,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
63995,10,0,3.500000,26000.000000,4,14,104000,False,True,False,True
63996,5,1,1.800000,17800.000000,5,9,89000,False,True,True,False
63997,6,1,4.000000,30500.000000,2,8,61000,False,True,True,False
63998,1,1,3.200000,21600.000000,5,16,108000,True,False,False,False


# 테스트 단위를 맞추기 위해서 scaling

In [178]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler

In [179]:
mms = MinMaxScaler()
mms.fit(X_train)
mms_X_train = mms.transform(X_train)
mms_X_valid = mms.transform(X_valid)
mms_X_test = mms.transform(X_test)

In [180]:
mms_X_train

array([[0.72727273, 1.        , 0.6       , ..., 0.        , 0.        ,
        1.        ],
       [0.18181818, 0.        , 0.48      , ..., 0.        , 0.        ,
        1.        ],
       [0.90909091, 1.        , 0.5       , ..., 1.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.26666667, ..., 0.        , 0.        ,
        1.        ],
       [0.90909091, 1.        , 0.44      , ..., 0.        , 0.        ,
        1.        ],
       [0.36363636, 1.        , 0.2       , ..., 1.        , 0.        ,
        1.        ]])

넘파이 어레이로 나오기 때문에 데이터 프레임으로 다시 변경

In [181]:
mms_X_train = pd.DataFrame(mms_X_train, columns=X_train.columns)
mms_X_valid = pd.DataFrame(mms_X_valid, columns=X_train.columns)
mms_X_test = pd.DataFrame(mms_X_test, columns=X_train.columns)

In [182]:
X_test

,recency,is_referral,mean_num_item,mean_total_amount,freq,sum_num_item,sum_total_amount,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
15576,12,1,5.000000,23000.0,1,5,23000,True,False,True,False
24159,4,1,2.333333,21000.0,3,7,63000,True,False,True,False
11135,2,1,2.500000,30500.0,2,5,61000,False,True,True,False
46485,5,0,2.250000,22000.0,4,9,88000,True,False,True,False
27143,6,1,2.500000,16250.0,4,10,65000,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
54788,11,1,4.000000,16600.0,5,20,83000,True,False,False,True
12199,4,0,2.000000,21600.0,5,10,108000,True,False,False,True
23992,4,0,2.800000,29800.0,5,14,149000,False,True,True,False
59787,7,1,4.200000,27400.0,5,21,137000,False,True,True,False


In [184]:
mms_X_test

,recency,is_referral,mean_num_item,mean_total_amount,freq,sum_num_item,sum_total_amount,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
0,1.000000,1.0,0.800000,0.500000,0.00,0.148148,0.085227,1.0,0.0,1.0,0.0
1,0.272727,1.0,0.266667,0.433333,0.50,0.222222,0.312500,1.0,0.0,1.0,0.0
2,0.090909,1.0,0.300000,0.750000,0.25,0.148148,0.301136,0.0,1.0,1.0,0.0
3,0.363636,0.0,0.250000,0.466667,0.75,0.296296,0.454545,1.0,0.0,1.0,0.0
4,0.454545,1.0,0.300000,0.275000,0.75,0.333333,0.323864,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
12795,0.909091,1.0,0.600000,0.286667,1.00,0.703704,0.426136,1.0,0.0,0.0,1.0
12796,0.272727,0.0,0.200000,0.453333,1.00,0.333333,0.568182,1.0,0.0,0.0,1.0
12797,0.272727,0.0,0.360000,0.726667,1.00,0.481481,0.801136,0.0,1.0,1.0,0.0
12798,0.545455,1.0,0.640000,0.646667,1.00,0.740741,0.732955,0.0,1.0,1.0,0.0


In [129]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.svm import SVC


In [185]:
knn = KNeighborsClassifier( n_jobs=8 )
knn.fit(mms_X_train, y_train)
valid_pred = knn.predict(mms_X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, valid_pred))

============================== valid_result ==============================
              precision    recall  f1-score   support

           0       0.87      0.96      0.91     10921
           1       0.39      0.14      0.21      1879

    accuracy                           0.84     12800
   macro avg       0.63      0.55      0.56     12800
weighted avg       0.80      0.84      0.81     12800



# KNN의 하이퍼파라미터 튜닝
* n_neighbors = ?
* 전체 샘플 수가 적을때 10000개 이하일 때: 3,5, 7같은 홀수값을 주로 넣어준다
* 전체 샘플수가 많을 때 10000개 초과일 때: 루트(n), log2(n)

In [186]:
len(mms_X_train)

38400

# 최적 k를 계산하면?

In [188]:
# 루트를 이용한 방법
n = len(mms_X_train)
print(n)
k = int(np.sqrt(n))
print(np.sqrt(n), k)

38400
195.95917942265424 195


In [189]:
# log2(n)를 이용한 최적 k 계산
k_log2 = int(np.log2(n))
print(k_log2 )

15


In [190]:
knn = KNeighborsClassifier(n_neighbors=195, n_jobs=8 )
knn.fit(mms_X_train, y_train)
valid_pred = knn.predict(mms_X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, valid_pred))

============================== valid_result ==============================
              precision    recall  f1-score   support

           0       0.85      1.00      0.92     10921
           1       0.00      0.00      0.00      1879

    accuracy                           0.85     12800
   macro avg       0.43      0.50      0.46     12800
weighted avg       0.73      0.85      0.79     12800



/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [191]:
knn = KNeighborsClassifier( n_neighbors=15, n_jobs=15 )
knn.fit(mms_X_train, y_train)
valid_pred = knn.predict(mms_X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, valid_pred))

============================== valid_result ==============================
              precision    recall  f1-score   support

           0       0.86      0.99      0.92     10921
           1       0.59      0.07      0.13      1879

    accuracy                           0.86     12800
   macro avg       0.73      0.53      0.52     12800
weighted avg       0.82      0.86      0.81     12800



데이터 증폭을 통한 성능개선

In [192]:
y.value_counts()

conversion
0    54606
1     9394
Name: count, dtype: int64

In [193]:
mms_X_train.columns

Index(['recency', 'is_referral', 'mean_num_item', 'mean_total_amount', 'freq',
       'sum_num_item', 'sum_total_amount', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Phone', 'channel_Web'],
      dtype='object')

In [194]:
from imblearn.over_sampling import SMOTENC

In [195]:
smtnc = SMOTENC(categorical_features=[1, 7, 8, 9, 10], random_state=42)
smt_X_train, smt_y_train = smtnc.fit_resample(mms_X_train, y_train)

In [196]:
smt_y_train.value_counts()

conversion
0    32764
1    32764
Name: count, dtype: int64

In [197]:
len(smt_X_train)


65528

In [198]:
	

n = len(smt_X_train)
k_log2 = int(np.log2(n))
print(k_log2)


15


In [199]:


# log2(n)를 이용해 계산한 k 값 15 적용
knn = KNeighborsClassifier(n_neighbors=15, n_jobs=8)
knn.fit(smt_X_train, smt_y_train)
valid_pred = knn.predict(mms_X_valid)
print(classification_report(y_valid, valid_pred))
print("========= Test Result=========")
test_pred = knn.predict(X_test)
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.91      0.67      0.77     10921
           1       0.24      0.60      0.34      1879

    accuracy                           0.66     12800
   macro avg       0.57      0.64      0.56     12800
weighted avg       0.81      0.66      0.71     12800

========= Test Result=========
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0.04     12800



/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [200]:
	

# log2(n)를 이용해 계산한 k 값 15 적용
for i in range(3, 22, 2):
    knn = KNeighborsClassifier(n_neighbors=i, n_jobs=8)
    knn.fit(smt_X_train, smt_y_train)
    valid_pred = knn.predict(mms_X_valid)
    print(classification_report(y_valid, valid_pred))
    print(f"========= Test Result {i}=========")
    test_pred = knn.predict(X_test)
    print(classification_report(y_test, test_pred))


              precision    recall  f1-score   support

           0       0.89      0.75      0.81     10921
           1       0.23      0.44      0.31      1879

    accuracy                           0.71     12800
   macro avg       0.56      0.60      0.56     12800
weighted avg       0.79      0.71      0.74     12800

========= Test Result 3=========


/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0.04     12800

              precision    recall  f1-score   support

           0       0.89      0.72      0.80     10921
           1       0.23      0.49      0.32      1879

    accuracy                           0.69     12800
   macro avg       0.56      0.61      0.56     12800
weighted avg       0.79      0.69      0.73     12800

========= Test Result 5=========


/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0.04     12800

              precision    recall  f1-score   support

           0       0.90      0.70      0.78     10921
           1       0.23      0.53      0.32      1879

    accuracy                           0.67     12800
   macro avg       0.56      0.61      0.55     12800
weighted avg       0.80      0.67      0.72     12800

========= Test Result 7=========


...

/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0.04     12800

              precision    recall  f1-score   support

           0       0.91      0.67      0.77     10921
           1       0.24      0.62      0.35      1879

    accuracy                           0.66     12800
   macro avg       0.58      0.64      0.56     12800
weighted avg       0.81      0.66      0.71     12800

========= Test Result 19=========


/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0.04     12800

              precision    recall  f1-score   support

           0       0.91      0.66      0.77     10921
           1       0.24      0.63      0.35      1879

    accuracy                           0.66     12800
   macro avg       0.58      0.64      0.56     12800
weighted avg       0.81      0.66      0.71     12800

========= Test Result 21=========
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0

/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [201]:

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight="balanced", n_jobs=8, random_state=42)
rfc.fit(smt_X_train, smt_y_train)
valid_pred = rfc.predict(mms_X_valid)
print("========= Valid Result=========")
print(classification_report(y_valid, valid_pred))
print("========= Test Result=========")
test_pred = rfc.predict(X_test)
print(classification_report(y_test, test_pred))
print()


========= Valid Result=========
              precision    recall  f1-score   support

           0       0.88      0.91      0.89     10921
           1       0.35      0.30      0.32      1879

    accuracy                           0.82     12800
   macro avg       0.62      0.60      0.61     12800
weighted avg       0.80      0.82      0.81     12800

========= Test Result=========
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     10921
           1       0.15      1.00      0.26      1879

    accuracy                           0.15     12800
   macro avg       0.07      0.50      0.13     12800
weighted avg       0.02      0.15      0.04     12800




/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jihyun/miniforge3/envs/fintech/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

# 서포트벡터 SVC를 사용해서 분석
* StandardScale
* 카테고리 변수를 제외하고 StandardScaler를 사용하는 것이 좋음

In [202]:
	

from sklearn.preprocessing import StandardScaler


In [215]:
X_train.columns

Index(['recency', 'is_referral', 'mean_num_item', 'mean_total_amount', 'freq',
       'sum_num_item', 'sum_total_amount', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Phone', 'channel_Web'],
      dtype='object')

In [216]:
X_train_cat = X_train[['is_referral', 'zip_code_Surburban', 'zip_code_Urban',
       'channel_Phone', 'channel_Web']]
X_train_num = X_train[['recency','mean_num_item', 'mean_total_amount', 'freq',
       'sum_num_item', 'sum_total_amount']]

In [217]:
num_cols = X_train_num.columns

In [222]:
X_valid.index

Index([61161, 55598, 25013, 19407, 30427, 35564, 36304, 38135,  6963, 52077,
       ...
       18342,   417, 24400, 27134, 56773, 53675, 54661, 36640, 30738,  5489],
      dtype='int64', length=12800)

In [223]:
X_train_num.index

Index([11734, 43876, 34934, 30562, 32242, 44194, 10988, 32324,  3065, 24950,
       ...
       45360, 27982,  2420,  3126, 56832, 44666, 29962,  4734,  3437, 27103],
      dtype='int64', length=38400)

In [221]:
X_train_num.columns

Index(['recency', 'mean_num_item', 'mean_total_amount', 'freq', 'sum_num_item',
       'sum_total_amount'],
      dtype='object')

In [218]:
X_train[num_cols]

,recency,mean_num_item,mean_total_amount,freq,sum_num_item,sum_total_amount
11734,9,4.000000,15000.000000,2,8,30000
43876,3,3.400000,28000.000000,5,17,140000
34934,11,3.500000,20500.000000,4,14,82000
30562,2,4.333333,24000.000000,3,13,72000
32242,7,1.000000,17000.000000,2,2,34000
...,...,...,...,...,...,...
44666,4,1.000000,9000.000000,1,1,9000
29962,2,2.500000,12500.000000,2,5,25000
4734,1,2.333333,20666.666667,3,7,62000
3437,11,3.200000,20800.000000,5,16,104000


In [229]:
ss = StandardScaler()
ss.fit(X_train_num)
X_train_num_temp = ss.transform(X_train_num)
X_valid_num_temp = ss.transform(X_train[num_cols])
X_test_num_temp = ss.transform(X_train[num_cols])


In [230]:
ss_X_train_num = pd.DataFrame(X_train_num_temp, columns=num_cols, index=X_train_num.index)
ss_X_valid_num = pd.DataFrame(X_valid_num_temp, columns=num_cols, index=X_valid.index)
ss_X_test_num = pd.DataFrame(X_test_num_temp, columns=num_cols, index=X_test.index)

ValueError: Shape of passed values is (38400, 6), indices imply (12800, 6)

In [124]:
ss_X_test_num

,recency,amt_sum,amt_count,amt_sum_zscore,amt_count_zscore,num_item
0,0.925126,-1.106445,-0.778084,-1.106445,-0.778084,-0.778084
1,-0.786076,2.158693,1.388747,2.158693,1.388747,1.388747
2,1.495527,0.437075,0.666470,0.437075,0.666470,0.666470
3,-1.071276,0.140244,-0.055807,0.140244,-0.055807,-0.055807
4,0.354725,-0.987713,-0.778084,-0.987713,-0.778084,-0.778084
...,...,...,...,...,...,...
38395,-0.500876,-1.729789,-1.500361,-1.729789,-1.500361,-1.500361
38396,-1.071276,-1.254860,-0.778084,-1.254860,-0.778084,-0.778084
38397,-1.356477,-0.156587,-0.055807,-0.156587,-0.055807,-0.055807
38398,1.495527,1.090102,1.388747,1.090102,1.388747,1.388747


In [227]:
cat_cols = X_train_cat.columns


In [224]:
X_train_cat.columns

Index(['is_referral', 'zip_code_Surburban', 'zip_code_Urban', 'channel_Phone',
       'channel_Web'],
      dtype='object')

In [228]:
	

ss_X_train = pd.concat([ss_X_train_num, X_train_cat], axis=1)
ss_X_valid = pd.concat([ss_X_valid_num, X_valid[cat_cols]], axis=1)
ss_X_test = pd.concat([ss_X_test_num, X_test[cat_cols]], axis=1)


In [231]:
from sklearn.svm import SVC

In [232]:
svc = SVC()
svc.fit(ss_X_train, y_train)
valid_pred = svc.predict(ss_X_valid)
print("========= Valid Result=========")
print(classification_report(y_valid, valid_pred))
print("========= Test Result=========")
test_pred = svc.predict(ss_X_test)
print(classification_report(y_test, test_pred))
print()


KeyboardInterrupt: 